## Import library

In [1]:
import numpy as np 
import pandas as pd 
from datetime import datetime
import copy as cp
import calendar
from datetime import timedelta

In [2]:
# 載入所需函示庫
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels
import statsmodels.stats.libqsturng
from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 150

In [3]:
def strtoint(s):
   #print(s)
    if type(s) != str :
        return s
    ss = s.split(',')
    #print(ss[0])
    temp = 0
    for i in range(len(ss)):
        temp = temp * 1000 + int(ss[i])
#     temp = float(temp / 1000000)
    return temp

## Build a library

In [4]:
transportCnt = dict()

In [5]:
year = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [6]:
for i in year:
    for j in month:
        if i == '2022' and j == '04':
            break
        dt_name = '+OÑ_¦¦¦BÑ¦¿t¦+«+¦B¦q¦+¡p_' + i + j+'.csv'
        df = pd.read_csv(dt_name,encoding='big5')
        for k in range(df.shape[0]):
            tmp = df['營運日'][k]
            if(tmp != tmp):
                continue
            tmp = df['營運日'][k].split('/')
            date = datetime(int(tmp[0]) + 1911, int(tmp[1]), int(tmp[2]))
            transport = strtoint(df['總運量'][k])
            transportCnt.setdefault(date,transport)

## Functions

In [7]:
def getHoliday(year, Holiday, df):
    name = []
    y = year
    i = year - 2005
    if(type(df[Holiday][i]) == datetime):
        name.append(df[Holiday][i])
    else:
        start, end = df[Holiday][i].split('-')
        start = start.split('/')
        start = [int(s) for s in start]
        end = end.split('/')
        end = [int(s) for s in end]

        if(start[0] == end[0]):
            name += [datetime(y, start[0], x) for x in np.arange(start[1], end[1] + 1, 1)]
        else:
            lastday = calendar.monthrange(y, start[0])[1]
            if(start[0] == 12):
                name += [datetime(y - 1, start[0], x) for x in np.arange(start[1], lastday + 1, 1)]
            else:
                name += [datetime(y, start[0], x) for x in np.arange(start[1], lastday + 1, 1)]
            name += [datetime(y, end[0], x) for x in np.arange(1, end[1] + 1, 1)]
    return name

In [8]:
def getHolidayLength(date):
    holiday, column, i = isHoliday([date.year], h_columns, date)
    if(holiday):
        return df[column][i]
    elif(date.isoweekday() == 6 or date.isoweekday() == 7):
        return 2
    else:
        return 0

In [9]:
def isHoliday(years, columns, date):
    for column in columns:
        for year in years:
            if(date in holiday_rec[column][year - 2005]):
                return True, column, year - 2005
    return False, -1, -1

In [10]:
def holiday_type(date):
    y = date.year
    m = date.month
    d = date.day
    traditional = ['春節', '端午', '中秋']
    national = ['雙十', '二二八', '元旦', '清明', '勞動']
    years = np.arange(2015, 2023, 1)
    if(isTyphoon(date) or ((m == 12) and (d == 31))):
        return float('nan')
    elif(isHoliday(years, traditional, date)[0]):
        return 'Traditional'
    elif(isHoliday(years, national, date)[0]):
        return 'National'
    elif((m == 12 and d == 25) or (m == 2 and d == 14) or (m == 10 and d == 31)):
        return 'Western'
    elif((m == 8 and d == 8) or (m == 5 and d <= 14 and d >= 8 and date.isoweekday() == 7)):
        return 'Fother'
    elif(date.isoweekday() == 6 or date.isoweekday() == 7):
        return 'Weekend'
    else:
        return 'Weekday'

In [11]:
def season(date):
    s = []
    start = 3
    for i in range( 4 ) : 
        s.append( [])
        for j in range( 3 ) :
            s[i].append( start ) 
            start = ( start % 12 ) + 1
    m = date.month
    if(m in s[0]):
        return 'Spring'
    elif(m in s[1]):
        return 'Summer'
    elif(m in s[2]):
        return 'Fall'
    else:
        return 'Winter'

In [40]:
def new_holiday_type1(date):
    holidays = [['除夕'], ['春節', '端午', '中秋', '清明'], ['雙十', '二二八', '元旦']]
    name = ['除夕', '三大', '國定']
    y = date.year
    m = date.month
    d = date.day
    years = np.arange(2015, 2023, 1)
    if(isTyphoon(date) or ((m == 12) and (d == 31))):
        return
    for i, h in enumerate(holidays):
        if(isHoliday(years, h, date)[0]):
            return name[i]
#     if((m == 12 and d == 25) or (m == 2 and d == 14) or (m == 10 and d == 31)):
#         return 'Western'
#     if(date.isoweekday() == 6 or date.isoweekday() == 7):
#         return '假日'
#     return '平日'


In [13]:
def isTyphoon(date):
    if(date in list(df_typhoon['日期'])):
        return True
    return False

## Global

In [14]:
df = pd.read_excel('../歷年連假天數.xlsx')
df = df.drop(index = 18)
df_typhoon = pd.read_excel('../Typhoon_date.xlsx')
columns = ['除夕', '春節.1', '端午.1', '中秋.1', '雙十.1', '二二八.1', '元旦.1', '清明.1', '勞動.1']
years = np.arange(2005, 2023, 1)
# type(df['除夕'][0]) == datetime
holiday_rec = []
for column in columns:
    tmp = []
    for year in years:
        tmp.append(getHoliday(year, column, df))
    holiday_rec.append(tmp)
holiday_rec = pd.DataFrame(holiday_rec).T
h_columns = ['除夕','春節', '端午', '中秋', '雙十', '二二八', '元旦', '清明', '勞動']
holiday_rec.columns = h_columns

--------

## T 檢定

In [59]:
days = [[]for i in range(2)]


In [60]:
year = np.arange(2015, 2020, 1)
month = np.arange(1, 13, 1)
for y in year:
    for m in month:
        for d in np.arange(1, calendar.monthrange(y,m)[1] + 1, 1):
            today = datetime(y,m,d)
            if(new_holiday_type1(today) == '三大'):
                days[0].append(transportCnt[today])
            elif(new_holiday_type1(today) == '國定'):
                days[1].append(transportCnt[today])

In [61]:
df1 = pd.DataFrame(days).T
columns = ['三大', '國定']
df1.columns = columns
df1

,三大,國定
0,1342552.0,1796950.0
1,1356261.0,1830184.0
2,1678593.0,1802037.0
3,1653840.0,1585463.0
4,1578871.0,2031991.0
...,...,...
72,1802883.0,NaN
73,1666067.0,NaN
74,1546868.0,NaN
75,1683910.0,NaN


In [62]:
df_num = len(df1['三大']) - 1
df_den = len(df1['國定']) - 1
alpha = 0.05
s1 = np.var(df1['三大'], ddof = 1)
s2 = np.var(df1['國定'], ddof = 1)
f_value = s1 / s2
print(f"F-value = {f_value}")
p_value = stats.f.cdf(f_value, df_num, df_den) * 2
print(f"p-value = {p_value}")

F-value = 0.7493944483373323
p-value = 0.21085700386156722


Since the p-value(0.2108) is greater than alpha(0.05), we cannot reject our null hypothesis. Therefore, we assume the two populations have similar variance and we may apply equal-variance t-test.

In [66]:
# t-test using sms.CompareMeans
cm = sms.CompareMeans(sms.DescrStatsW(df1['三大']), sms.DescrStatsW(df1['國定'].dropna()))
# usevarstr, ‘pooled’ or ‘unequal’
cm.summary(usevar='pooled', alpha=0.05)

,coef,std err,t,P>|t|,[0.025,0.975]
subset #1,-2.142e+05,3.2e+04,-6.703,0.000,-2.78e+05,-1.51e+05
